# ADS1115

## Accuracy
- 16-bit ADC
### Voltage measurement range (+/-)
- 6.144 V
- 4.096 V
- 2.048 V
- 1.024 V
- 0.512 V
- 0.256 V

## Impedance
- Common mode: 
	- Impedance measured by shorting pins and measuring current flowing through the 2 pins
	- 100 MOhm for 0.512 V, 0.256 V
	- 3 MOhm for 1.024 V
	- 6 MOhm for 2.048 V
- Differential input impedance:
	- Impedance measured by applying differential voltage, and measuring current from one pin to another
	- 710 kOhm for 0.512 V, 0.256 V
	- 2.4 MOhm for 1.024 V
	- 4.9 MOhm at 2.048 V

Guess: 
- The differential impedance is the one that matters.
- This is the impedance measured between the 2 inputs.


## Fsample
- 2**3 (8) SPS
- 2**4 (16) SPS
- ..
- 2**7 (128) SPS
- 250 SPS
- 475 SPS
- 860 SPS

## ESD protection
- ESD diodes are integrated to VDD and GND


## Differential vs Single-Ended
It seems that you can choose
- Either 2 differential inputs
- Either 4 single-ended inputs

There's MUXing going on where channels can be configured while the ADC is running to be 
- Differential
- Single ended

Measurements to be done:
- Measure the VBATT voltage
- Measure the Current going to the FPGA
- Measure the total current coming from the connector -> Buffer and turn into single-ended
- Measure the current going to VDC (inverter) differentially -> Buffer and turn into single-ended


## Required signal gain
- Current measurement resistors: 0.002 mOhm
- Common-mode input impedance: 100 MOhm (FSR  +-0.256 mV)

### ITOT Measurement
- Itot: 0.1 - 5 A

In [7]:
R_current = 0.002
Ampl = 20
# Current
I_min = 0.1
I_max = 5

# Voltage
V_Itot_min = I_min*R_current * Ampl
V_Itot_max = I_max*R_current * Ampl

# P_max
P_max = R_current * I_max * I_max


print(f"Vrange: {V_Itot_min} - {V_Itot_max}")
print(f"P_max: {P_max}")

Vrange: 0.004 - 0.2
P_max: 0.05


### IDC Measurement
- I_dc: 0 - 5 A

In [8]:
R_current = 0.002

Ampl = 20
# Current
I_min = 0.1
I_max = 5

# Voltage
V_Itot_min = I_min*R_current * Ampl
V_Itot_max = I_max*R_current * Ampl

# P_max
P_max = R_current * I_max * I_max


print(f"Vrange: {V_Itot_min} - {V_Itot_max}")
print(f"P_max: {P_max}")

Vrange: 0.004 - 0.2
P_max: 0.05


### IFPGA Measurement
- I_fpga: 0 - 2 A

In [9]:
R_current = 0.002

Ampl = 50
# Current
I_min = 0.1
I_max = 2

# Voltage
V_Ifpga_min = I_min*R_current * Ampl
V_Ifpga_max = I_max*R_current * Ampl

# P_max
P_max = R_current * I_max * I_max

print(f"Vrange: {V_Ifpga_min} - {V_Ifpga_max}")
print(f"P_max: {P_max}")

Vrange: 0.01 - 0.2
P_max: 0.008


### VBATT Measurement
- Vbatt: 0 -> 25 V
- Vmeas: +-256 mV .. +- 6.144 V

In [6]:
Vbat_min = 0.1
Vbat_max = 25
Vin_max = 2.048
div = Vin_max / Vbat_max

print(f"Vrange: [{div * Vbat_min}, {div * Vbat_max}]")

# Thevenin equivalent impedance calculation
R2_div = 1800
R1_div = (1/div) * R2_div - R2_div
Rth_source = 1/((1/R1_div) + (1/R2_div))
print(f"R2_div: {R2_div}, R1_div: {R1_div}, Rth_source: {round(Rth_source,3)} Ohm")
R1 = 20e3
R2 = R2_div
check = (R2 / (R1+R2))
print(f"Check: {check} == {div}")

I_max = (Vbat_max / (R2+R1))
P_max = R1_div * I_max * I_max
print(f"P_max: {P_max*1e3} mW")

Vrange: [0.008192000000000001, 2.048]
R2_div: 1800, R1_div: 20172.656249999996, Rth_source: 1652.544 Ohm
Check: 0.08256880733944955 == 0.08192
P_max: 26.529564338544734 mW


## HV-problems

### Issue
Current measurements are easier done between supply and ground, in-circuit current measurements need to be isolated, since 
- OPAMPS can't handle voltages above their Vcc.
- Measuring raw voltage would give too high of a voltage drop for the voltage to be measured.

Solution: using a current-sensor. (Hall-effect sensor)

## ACS711
- Output voltage: 
	- Vuoh: Vcc-0.3 (so Vcc = 3V3)
	- viol: 0.3 V
- Sensitivity: 90 mV/A
	- 5 A: 450 mV
	- 2 A: 180 mV

### What is the conductivity when no power supplied
- Should be full, since the positive and negative are simply connected
	- Hall sensor simply measures the induced EMF through the high current

### Bandwidth of the voltage signal
- 100 kHz
	- Perhaps some low-pass filtering is in order to filter-out high frequency noise.
	- Simple RC-filter

### Source impedance of the Vout signal indicating the current?
- Supposed to be buffered according ot the block diagram.
- Impedances
	- R_load: 15 kOhm
	- C_load: 1 nF (maximum)

In [11]:
import math
R_load = 15e3 # Added as external resistor
C_load = 1e-9 # Added as external capacitor
# So RC filter pole
fc = 1/(2*math.pi * R_load * C_load)

print(f"LP fc: {round(fc*1e-3, 2)} kHz")

LP fc: 10.61 kHz


## Issue
It seems like the source impedance might be slightly low. Consider changing the hall-effect sensor so
- We can select a higher output voltage -> higher ADC input impedance
- We can have a lower source impedance

## Solution: TMCS1108
- Load impedance virtually 0 (0.2 - 2 Ohms)
- Max capacitive load: 1 nF
- 90 mA short-circuit current

### Add an RC filter here for noise filtering and current limiting

- $fc = 1/(2*\pi*R*C) = 1e3$
- $1 / (R*C)= 6.3 k\Omega$

In [26]:
import math
fc = 1e3
RC_prod = 1 / (2 * math.pi * 1e3)
print(f"RC_Prod: {RC_prod}")
R_temp = 1e3
C_temp = RC_prod / R_temp
print(f"R: {R_temp*1e-3} kOhm, C: {C_temp * 1e9} nF")
R_choice = 1e3
C_choice = 220e-9
print(f"R_choice: {R_choice*1e-3} kOhm, C_choice: {C_choice * 1e9} nF, f_cutoff = {round(1/(2*math.pi * R_choice * C_choice) * 1e-3, 2)} kHz")

RC_Prod: 0.00015915494309189535
R: 1.0 kOhm, C: 159.15494309189535 nF
R_choice: 1.0 kOhm, C_choice: 220.0 nF, f_cutoff = 0.72 kHz
